In [1]:
import pandas as pd
import yfinance as yf
import datetime as dt
import streamlit as st
from datetime import datetime
import os
from operator import itemgetter, attrgetter
import numpy as np

import streamlit as st
import plotly.express as px



import warnings    
warnings.filterwarnings("ignore")





/Users/familialima/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
#######################################################################################################
                            ######### Carga do Dataframe ##########
#######################################################################################################
file = '/Outputs/MinhasCotacoes.xlsx'
caminho_atual = os.getcwd()
file = caminho_atual + file

In [3]:
minhasacoes = pd.read_excel(file, sheet_name='Cotação das Ações')

In [4]:
def retorna_ultima_cotacao_acao(df):
    latest_quotes = df.loc[df.groupby('ACAO')['DATACOTACAO'].idxmax()]
    latest_quotes = latest_quotes[['ACAO', 'DATACOTACAO', 'QTDECOTAS', 'VALORFECHAMENTO']].drop_duplicates()
    latest_quotes['VALORTOTALATUAL'] = latest_quotes['QTDECOTAS'] * latest_quotes['VALORFECHAMENTO']
    return latest_quotes

In [5]:
def create_dataframe(coluna):
    df = minhasacoes[[coluna, 'QTDECOTAS', 'VALORCOMPRA', 'ACAO']].drop_duplicates()
    df['VALORTOTALCOMPRA'] = df['QTDECOTAS'] * df['VALORCOMPRA']
    df = df.groupby([coluna, 'ACAO']).agg({
        'QTDECOTAS': np.sum,
        'VALORTOTALCOMPRA': np.sum
    }).sort_values(by='VALORTOTALCOMPRA', ascending=False).reset_index()
    return df

In [6]:
def consolidate_details(df, latest_quotes):
    df = pd.merge(df, latest_quotes[['ACAO', 'DATACOTACAO', 'VALORTOTALATUAL']], on='ACAO', how='inner')
    df = df.drop_duplicates().sort_values(by='ACAO').reset_index(drop=True)
    return df

In [7]:
def group_by(df,coluna):
    df = df.groupby(coluna).agg({
        'QTDECOTAS': np.sum,
        'VALORTOTALCOMPRA': np.sum,
        'VALORTOTALATUAL': np.sum
    }).sort_values(by='VALORTOTALCOMPRA', ascending=False).reset_index()
    return df

In [8]:
def update_performance(df):
    df['RENDIMENTO(%)'] = ((df['VALORTOTALATUAL'] / df['VALORTOTALCOMPRA'] - 1) * 100).round(2)
    df['PREJUIZO(%)'] = ((df['VALORTOTALCOMPRA'] / df['VALORTOTALATUAL'] - 1) * 100).round(2)
    df.loc[df['VALORTOTALATUAL'] < df['VALORTOTALCOMPRA'], 'RENDIMENTO(%)'] = 0
    df.loc[df['VALORTOTALATUAL'] >= df['VALORTOTALCOMPRA'], 'PREJUIZO(%)'] = 0
    return df

In [9]:
def agupamento(agrupamento, df):
    coluna = agrupamento
    ult_cot = retorna_ultima_cotacao_acao(df)
    dtf = create_dataframe(coluna)
    dtf = consolidate_details(dtf, ult_cot)
    dtf = group_by(dtf, coluna)
    dtf = update_performance(dtf)
    return dtf

In [10]:
st = agupamento('SETORECONOMICO', minhasacoes)
subst = agupamento('SUBSETOR', minhasacoes)
sg = agupamento('SEGMENTO', minhasacoes)
emp = agupamento('EMPRESA', minhasacoes)

In [11]:
st

,SETORECONOMICO,QTDECOTAS,VALORTOTALCOMPRA,VALORTOTALATUAL,RENDIMENTO(%),PREJUIZO(%)
0,"Petróleo, Gás e Biocombustíveis",464,9465.68,18198.32,92.26,0.00
1,Financeiro,486,7675.54,5469.64,0.00,40.33
2,Consumo Cíclico,488,6933.29,2334.64,0.00,196.97
3,Saúde,213,5202.53,1558.36,0.00,233.85
4,Comunicações,2740,3251.00,3377.20,3.88,0.00
5,Materiais Básicos,247,3025.79,4505.60,48.91,0.00
6,Consumo não Cíclico,101,2001.04,1287.64,0.00,55.40
7,Bens Industriais,107,1293.33,832.72,0.00,55.31
8,Utilidade Pública,100,1251.00,1131.00,0.00,10.61
9,Tecnologia da Informação,50,1023.50,230.50,0.00,344.03


In [16]:
fig = px.bar(st, x="VALORTOTALCOMPRA", y="SETORECONOMICO", orientation='h')
fig.show()

In [15]:
import plotly.express as px

df = px.data.gapminder().query("country == 'Canada'")
fig = px.bar(df, x='year', y='pop',
             hover_data=['lifeExp', 'gdpPercap'], color='lifeExp',
             labels={'pop':'population of Canada'}, height=400)
fig.show()

In [13]:
df

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
240,Canada,Americas,1952,68.750,14785584,11367.16112,CAN,124
241,Canada,Americas,1957,69.960,17010154,12489.95006,CAN,124
242,Canada,Americas,1962,71.300,18985849,13462.48555,CAN,124
243,Canada,Americas,1967,72.130,20819767,16076.58803,CAN,124
244,Canada,Americas,1972,72.880,22284500,18970.57086,CAN,124
245,Canada,Americas,1977,74.210,23796400,22090.88306,CAN,124
246,Canada,Americas,1982,75.760,25201900,22898.79214,CAN,124
247,Canada,Americas,1987,76.860,26549700,26626.51503,CAN,124
248,Canada,Americas,1992,77.950,28523502,26342.88426,CAN,124
249,Canada,Americas,1997,78.610,30305843,28954.92589,CAN,124
